In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

In [ ]:
using Pkg
Pkg.activate("..")

: 

## Library downloads

In [ ]:
using JuMP,Ipopt,MosekTools, PyPlot
ipopt = () -> Ipopt.Optimizer()
mosek = () -> Mosek.Optimizer()

: 

## Data Structure and auxiliary functions

In [ ]:
include("../src/structs.jl")
include("../src/data_read.jl")
include("../src/utils.jl")

: 

## Data Download

Data library: [pglib-opf-master](https://github.com/power-grid-lib/pglib-opf)

In [ ]:
full_data = raw"..\data\pglib-opf"

: 

# Model build

## Functions related to electrical grid graph

Consider a non-directed graph with $\mathcal{N} = (\mathcal{B},\mathcal{L})$, where $\mathcal{B}$ is the set of buses and $\mathcal{L}$ the set of transmission lines. The set of Generators $\mathcal{G}$ is a subset of $\mathcal{B}$.

In [ ]:
function bus_id(Buses)
    count = 1
    dict_buses = Dict()
    for bus in Buses
        dict_buses[bus.bus_i] = count
        count += 1
    end
    dict_buses
end

: 

The function $\delta$ receive a bus as attribute and returns all lines connected to it.

In [ ]:
function δ(i,Branches,Buses)
    tbuses = []
    dict_buses = bus_id(Buses)
    count = 1
    ls = []
    for branch in Branches
        if dict_buses[branch.fbus] == i
            push!(tbuses,dict_buses[branch.tbus])
            push!(ls,count)
        end
        if dict_buses[branch.tbus] == i
            push!(tbuses,dict_buses[branch.fbus])
            push!(ls,count)
        end
        count += 1
    end
    (tbuses,ls)
end

: 

In [ ]:
function From_nodes(i,Branches,Buses)
    tbuses = []
    ls = []
    count = 1
    dict_buses = bus_id(Buses)
    for branch in Branches
        if dict_buses[branch.fbus] == i
            push!(tbuses,dict_buses[branch.tbus])
            push!(ls,count)
        end
        count += 1
    end
    (tbuses,ls)
end

function To_nodes(i,Branches,Buses)
    tbuses = []
    ls = []
    count = 1
    dict_buses = bus_id(Buses)
    for branch in Branches
        if dict_buses[branch.tbus] == i
            push!(tbuses,dict_buses[branch.fbus])
            push!(ls,count)
        end
        count += 1
    end
    (tbuses,ls)
end

: 

In [ ]:
function G(i,Generators,Buses)
    G_i = []
    count = 1
    dict_buses = bus_id(Buses)
    for gen in Generators
        if dict_buses[gen.bus] == i
            push!(G_i,count)
        end
        count += 1
    end
    G_i
end

: 

## Constants

- $Y_l^{Re}$: Series admittance (real)
- $Y_l^{Im}$: Series admittance (imaginary)
- $T_l^{Re}$: Transformer parameters (real)
- $T_l^{Im}$: Transformer parameters (imaginary)
- $b_l$: Line charge
- $s_l^u$: Thermal limits
- $\theta_{min}$,$\theta_{max}$: Branch voltage angle difference range

In [ ]:
function add_constants(Branches)
    YR = zeros(length(Branches))
    YI = zeros(length(Branches))
    TR = zeros(length(Branches))
    TI = zeros(length(Branches))
    B = zeros(length(Branches))
    thermal_lim = zeros(length(Branches))
    ang_min = zeros(length(Branches))
    ang_max = zeros(length(Branches))
    for l=1:length(Branches)
        YR[l] = Branches[l].r/(Branches[l].r^2+Branches[l].x^2)
        YI[l] = -Branches[l].x/(Branches[l].r^2+Branches[l].x^2)
        TR[l] = Branches[l].ratio*cos(Branches[l].angle)
        TI[l] = Branches[l].ratio*sin(Branches[l].angle)
        B[l] = Branches[l].b
        thermal_lim[l] = Branches[l].rateA/100.0
        ang_min[l] = Branches[l].angmin*pi/180
        ang_max[l] = Branches[l].angmax*pi/180
    end
    return YR,YI,TR,TI,B,thermal_lim,ang_min,ang_max
end

: 

## Unrelaxed variables

- $p_g^i \hspace{0.2cm} \forall i \in \mathcal{B}$: Power Output (active)
- $q_g^i \hspace{0.2cm} \forall i \in \mathcal{B}$: Power Output (reactive)
- $S^{Re}_{i,j,l} \hspace{0.2cm} \forall (i,j,l) \in \mathcal{L}$: Power flow on line (active)
- $S^{Im}_{i,j,l} \hspace{0.2cm} \forall (i,j,l) \in \mathcal{L}$: Power flow on line (reactive)

In [ ]:
function add_global_variables!(model, Buses, Generators, Branches, thermal_lim; start=false)
    if start
        arr1 = ones(length(Buses))
        arr2 = ones(length(Generators))
        arr3 = ones(length(Branches))
        @variable(model, Generators[i].Pmin/100.0 <= p_g[i=1:length(Generators)] <= Generators[i].Pmax/100.0, start = arr2[i])
        @variable(model, Generators[i].Qmin/100.0 <= q_g[i=1:length(Generators)] <= Generators[i].Qmax/100.0, start = arr2[i])
        SR = @variable(model,[(i,j,l) in [(i,j,l) for i = 1:length(Buses) for (j,l) in zip(δ(i,Branches,Buses)...)]], base_name = "SR",lower_bound = -thermal_lim[l], upper_bound = thermal_lim[l], start = arr3[l])
        SI = @variable(model,[(i,j,l) in [(i,j,l) for i = 1:length(Buses) for (j,l) in zip(δ(i,Branches,Buses)...)]], base_name = "SI",lower_bound = -thermal_lim[l], upper_bound = thermal_lim[l], start = arr3[l])
    else
        @variable(model, Generators[i].Pmin/100.0 <= p_g[i=1:length(Generators)] <= Generators[i].Pmax/100.0)
        @variable(model, Generators[i].Qmin/100.0 <= q_g[i=1:length(Generators)] <= Generators[i].Qmax/100.0)
        SR = @variable(model,[(i,j,l) in [(i,j,l) for i = 1:length(Buses) for (j,l) in zip(δ(i,Branches,Buses)...)]], base_name = "SR",lower_bound = -thermal_lim[l], upper_bound = thermal_lim[l])
        SI = @variable(model,[(i,j,l) in [(i,j,l) for i = 1:length(Buses) for (j,l) in zip(δ(i,Branches,Buses)...)]], base_name = "SI",lower_bound = -thermal_lim[l], upper_bound = thermal_lim[l])
    end
    model[:SR] = SR
    model[:SI] = SI
end

: 

## Voltage Variables

- $V_i^{Re} \hspace{0.2cm} \forall i \in \mathcal{B}$: Bus Voltage (active)
- $V_i^{Im} \hspace{0.2cm} \forall i \in \mathcal{B}$: Bus Voltage (reactive)

In [ ]:
function add_voltage_variables!(model, Buses; start = false)
    if start
        arr1 = ones(length(Buses))
        @variable(model, - Buses[i].Vmax <= VR[i=1:length(Buses)] <= Buses[i].Vmax , start = arr1[i])
        @variable(model, - Buses[i].Vmax <= VI[i=1:length(Buses)] <= Buses[i].Vmax , start = arr1[i])
    else
        @variable(model, - Buses[i].Vmax <= VR[i=1:length(Buses)] <= Buses[i].Vmax)
        @variable(model, - Buses[i].Vmax <= VI[i=1:length(Buses)] <= Buses[i].Vmax)
    end
end

: 

## Constraints

### Reference angle
- Constraint:
    - $\angle V_I(1) = 0$

In [ ]:
function add_reference_angle!(model, Buses)
    VI = variable_by_name(model, "VI[1]")
    @constraint(model, VI == 0)
end

: 

### Power Balance
- Real and Imaginary part of a complex number: $\Re(\cdot),\Im(\cdot)$ 
- Constant Power Demand on bus i (active)  : $P^d_i$
- Constant Power Demand on bus i (reactive): $Q^d_i$
- Bus Shunt Admittance: $Y^s_i$
- Constraints:
    - $\sum_{i \in G(i)} p_g^i - P^d_i + \Re(Y^s_i) |V_i|^2 = \sum_{(l,j) \in \delta(i)} S_{lij}^{Re} \hspace{0.2cm}  \forall i \in \mathcal{B}$ (active)
    - $\sum_{i \in G(i)} q_g^i - Q^d_i + \Im(Y^s_i) |V_i|^2 = \sum_{(l,j) \in \delta(i)} S_{lij}^{Re} \hspace{0.2cm}  \forall i \in \mathcal{B}$ (reactive)

In [ ]:
function constraint_power_balance!(model, Buses, Generators, Branches, sqrd_volt, i)
    p_g = model[Symbol("p_g")]
    q_g = model[Symbol("q_g")]
    SR = model[Symbol("SR")]
    SI = model[Symbol("SI")]
    @constraint(model, sum(p_g[i] for i in G(i,Generators,Buses)) - Buses[i].Pd/100.0 + Buses[i].Gs/100.0*sqrd_volt == sum(SR[(i,j,l)] for (j,l) in zip(δ(i,Branches,Buses)...)))
    @constraint(model, sum(q_g[i] for i in G(i,Generators,Buses)) - Buses[i].Qd/100.0 + Buses[i].Bs/100.0*sqrd_volt == sum(SI[(i,j,l)] for (j,l) in zip(δ(i,Branches,Buses)...)))
end

: 

### Voltage Bounds
- $v_i^l,v_i^u$: Lower and upper voltages limits
- Constraints:
    - $(v_i^l)^2 \leq |V_i|^2 \leq (v_i^u)^2 \hspace{0.2cm} \forall i \in \mathcal{B}$

In [ ]:
function voltage_bounds_sqrd!(model, Buses, sqrd_volt, i)
    @constraint(model, Buses[i].Vmin^2 <= sqrd_volt <= Buses[i].Vmax^2)
end

: 

### Line Balance
- Constraints:
    - $S_{lij}^{Re} = Y_l^{Re} |V_i|^2 - Y_l^{Re}c_{ij} - Y_l^{Im}s_{ij} \hspace{0.2cm} \forall (l,i,j) \in \mathcal{L}$
    - $S_{lij}^{Im} = -(Y_l^{Im}-b_l/2) |V_i|^2 + Y_l^{Im}c_{ij} - Y_l^{Re}s_{ij} \hspace{0.2cm} \forall (l,i,j) \in \mathcal{L}$
    - $c_{ij} = V_i^{Re}V_j^{Re}+V_i^{Im}V_j^{Im}$
	- $s_{ij} = V_i^{Im}V_j^{Re}-V_i^{Re}V_j^{Im}$

In [ ]:
function constraint_line_balance!(model, YR, YI, B, sqrd_volt_i, sqrd_volt_j, sum_product_voltages,  diff_product_voltages, i, j, l)
    SR = model[Symbol("SR")]
    SI = model[Symbol("SI")]
    
    @constraint(model, SR[(i,j,l)] == YR*sqrd_volt_i + (-YR)*sum_product_voltages + (-YI)*(diff_product_voltages))
            
    @constraint(model, SI[(i,j,l)] == -(YI+B/2)*sqrd_volt_i - (-YI)*sum_product_voltages + (-YR)*diff_product_voltages)

    @constraint(model, SR[(j,i,l)] == YR*sqrd_volt_j + (-YR)*sum_product_voltages + (-YI)*(-(diff_product_voltages)))

    @constraint(model, SI[(j,i,l)] == -(YI+B/2)*sqrd_volt_j - (-YI)*sum_product_voltages + (-YR)*(-(diff_product_voltages)))
end

: 

### Line thermal limit

- Constraints:
    - $(S_{lij}^{Re})^2 + (S_{lij}^{Im})^2 \leq (s_l^u)^2 \hspace{0.2cm} \forall (l,i,j) \in \mathcal{L}$

In [ ]:
function line_thermal_bounds_sqrd!(model,thermal_lim, i, j, l)
    SR = model[Symbol("SR")]
    SI = model[Symbol("SI")]
    
    @constraint(model, SI[(i,j,l)]^2+SR[(i,j,l)]^2 <= thermal_lim[l]^2)
    @constraint(model, SI[(j,i,l)]^2+SR[(j,i,l)]^2 <= thermal_lim[l]^2)
end

: 

### Phase Angle Differences

- Constraints:
    - $ c_{ij} \tan(\theta_{min}) \leq s_{ij} \leq c_{ij} \tan(\theta_{max})$
    - $c_{ij} = V_i^{Re}V_j^{Re}+V_i^{Im}V_j^{Im}$
	- $s_{ij} = V_i^{Im}V_j^{Re}-V_i^{Re}V_j^{Im}$

In [ ]:
function constraint_phase_angle_diff!(model, ang_max, ang_min, sum_product_voltages,  diff_product_voltages, i, j, l) 
    @constraint(model, diff_product_voltages <= tan(ang_max[l])*sum_product_voltages)
    @constraint(model, diff_product_voltages >= tan(ang_min[l])*sum_product_voltages)
end

: 

## Objective Function

### Generator fuel cost minimization
- $\min \sum_{k \in \mathcal{G}} c_{2k} (p_k^g)^2 + c_{1k} p_k^g + c_{0k}$

In [ ]:
function add_objective!(model, GeneratorCosts)
    p_g = model[Symbol("p_g")]
    q_g = model[Symbol("q_g")]
    obj = sum([GeneratorCosts[k].c2*p_g[k]^2*100.0^2+GeneratorCosts[k].c1*p_g[k]*100.0+GeneratorCosts[k].c0 for k = 1:length(GeneratorCosts)])
    @objective(model, Min, obj)
end

: 

# Convex Relaxations

Consider the complex matrix $X \in \mathbb{R}^{\mathcal{|B|}\times \mathcal{|B|}}$
- $X = V(V^T)^*$
- $X_{ij} = V_iV_j^* \hspace{0.2cm} \forall (i,j) \in \mathcal{B}^2$

$X$ is positive semidefinite matrix with rank $1$.

We can substitute the complex matrix to a real matrix $W \in \mathbb{R}^{2\mathcal{|B|}\times 2 \mathcal{|B|}}$, with the following transformations:

- $W_{ij}   = V_i^{Re}V_j^{Re} \hspace{0.2cm} \forall (i,j) \in \mathcal{B}^2$
- $W_{i'j'} = V_i^{Im}V_j^{Im} \hspace{0.2cm} \forall (i,j) \in \mathcal{B}^2$
- $W_{i'j}  = V_i^{Im}V_j^{Re} \hspace{0.2cm} \forall (i,j) \in \mathcal{B}^2$
- $W_{ij'}  = V_i^{Re}V_j^{Im} \hspace{0.2cm} \forall (i,j) \in \mathcal{B}^2$

Where $i' = i + |\mathcal{B}|$ and $j' = i + |\mathcal{B}|$.

$W$ is positive semidefinite matrix with rank $1$.

## McCormick Envelopes

The McCormick envelopes are the convex hull of the set $\{(x,y,w) \hspace{0.1cm} | \hspace{0.1cm} w = xy, (x,y) \in [\underline{x}, \bar{x}] \times [\underline{y}, \bar{y}]\}$, which are:
- $MC(w = xy) = \{(x, y, w): \max \{\underline{y} x+\underline{x} y-\underline{x} \underline{y}, \bar{y} x+\bar{x} y-\overline{x y}\} \leq w \leq \min \{\underline{y} x+\bar{x} y-\bar{x} \underline{y}, \bar{y} x+\underline{x} y-\underline{x} \bar{y}\}\}$

### Function to make McCormick envelopes

In [ ]:
function MC_Envelopes!(model,w,x,y)
    x_up = upper_bound(x) 
    x_down = lower_bound(x)
    y_up = upper_bound(y) 
    y_down = lower_bound(y)
    @constraint(model, y_down*x+x_down*y-x_down*y_down <= w)
    @constraint(model, w <= y_down*x+x_up*y-x_up*y_down)
    @constraint(model, y_up*x+x_up*y-x_up*y_up <= w)
    @constraint(model, w <= y_up*x+x_down*y-x_down*y_up)
end

: 

### Relaxation with McCormick Envelopes

- $MC(W_{ij}   = V_i^{Re}V_j^{Re}) \hspace{0.2cm} \forall (i,j) \in \mathcal{B}^2$
- $MC(W_{i'j'} = V_i^{Im}V_j^{Im}) \hspace{0.2cm} \forall (i,j) \in \mathcal{B}^2$
- $MC(W_{i'j}  = V_i^{Im}V_j^{Re}) \hspace{0.2cm} \forall (i,j) \in \mathcal{B}^2$
- $MC(W_{ij'}  = V_i^{Re}V_j^{Im}) \hspace{0.2cm} \forall (i,j) \in \mathcal{B}^2$

In [ ]:
function MC_relaxation!(model,Buses,Branches)
    VR = model[Symbol("VR")]
    VI = model[Symbol("VI")]
    @variable(model, W[i=1:2*length(Buses),j=1:2*length(Buses)])
    s = x -> x+length(Buses)
    sqrd_volt = [W[i,i]+W[s(i),s(i)] for i=1:length(Buses)]
    for i=1:length(Buses)
        MC_Envelopes!(model, W[i,i], VR[i], VR[i])
        MC_Envelopes!(model, W[s(i),s(i)], VI[i], VI[i])
    end
    for i=1:length(Buses)
        for (j,l) in zip(From_nodes(i,Branches,Buses)...)    
            MC_Envelopes!(model,W[i,j]      ,VR[i],VR[j])
            MC_Envelopes!(model,W[s(i),s(j)],VI[i],VI[j])
            MC_Envelopes!(model,W[s(i),j]   ,VI[i],VR[j])
            MC_Envelopes!(model,W[j,i]      ,VR[j],VR[i])
            MC_Envelopes!(model,W[s(j),s(i)],VI[j],VI[i])
            MC_Envelopes!(model,W[j,s(i)]   ,VR[i],VI[j])
        end
    end
end

: 

## Semidefinite Relaxation

- The rank $1$ restriction of $W$ matrix is non-convex, so we remove this constraint.

In [ ]:
function SDP_relaxation!(model,Buses)
    @variable(model, W[i=1:2*length(Buses),j=1:2*length(Buses)],PSD)
end

: 

## Second Order Cone Relaxation

The SOCP relaxation consists in relaxing the equalitry $X = V(V^*)^T$ to an inequality

$\hspace{0.5cm} \begin{array}{l}
	X_{i j}=V_{i} V_{j}^{*} \\
	X_{i j} X_{i j}^{*}=V_{i} V_{j}^{*} V_{i}^{*} V_{j} \\
	\left|X_{i j}\right|^{2}=X_{i i} X_{j j} \\
	\left|X_{i j}\right|^{2} \leqslant X_{i i} X_{j j}
	\end{array}$

Writing as a second order cone:

$\hspace{0.5cm} \left|\left(\begin{array}{c}
		2 X_{i j} \\
		X_{i i}-X_{j j}
		\end{array}\right) \right| \leqslant X_{i i}+X_{j j}$
        
In function of $W$:

$\hspace{0.5cm} \left|\left(\begin{array}{c}
2(W_{i j}+W_{i' j'}) \\
W_{i i}+W_{i' i'}-W_{j j}-W_{j' j'}
\end{array}\right) \right| \leqslant W_{i i}+W_{i' i'}+W_{j j}+W_{j' j'}$

In [ ]:
function SOCP_relaxation!(model,Buses,Branches)
    @variable(model, W[i=1:2*length(Buses),j=1:2*length(Buses)])
    s = x -> x+length(Buses)
    for i=1:length(Buses)
        for (j,l) in zip(From_nodes(i,Branches,Buses)...)
            @constraint(model, [W[i,i]+W[s(i),s(i)]+W[j,j]+W[s(j),s(j)], 2*(W[i,j]+W[s(i),s(j)]), W[i,i]+W[s(i),s(i)]-W[j,j]-W[s(j),s(j)]] in SecondOrderCone())
        end
    end
end

: 

# Model

In [ ]:
function build_problem(file::String, optimizer, relaxation::RELAXATION)
    Buses, Generators, GeneratorCosts, Branches = read_matlab_file(file);
    YR,YI,TR,TI,B,thermal_lim,ang_min,ang_max = add_constants(Branches)
    model = Model(optimizer)
    if relaxation == NL
        add_global_variables!(model, Buses, Generators, Branches, thermal_lim, start = true)
    else
        add_global_variables!(model, Buses, Generators, Branches, thermal_lim)
    end
    add_objective!(model, GeneratorCosts)
    
    if relaxation == NL 
        add_voltage_variables!(model, Buses, start = true)
    elseif relaxation == MC
        add_voltage_variables!(model, Buses)
    end
    if relaxation == NL || relaxation == MC
        add_reference_angle!(model, Buses)
        VR = model[Symbol("VR")]
        VI = model[Symbol("VI")]
    end
    
    if relaxation == NL
        sqrd_volt = VR.^2 .+ VI.^2
    end
    
        
    if relaxation == MC
        MC_relaxation!(model,Buses,Branches)
    end
    if relaxation == SDP
        SDP_relaxation!(model,Buses)
    end
    if relaxation == SOCP
        SOCP_relaxation!(model,Buses,Branches)
    end
    
    if relaxation == MC || relaxation == SDP || relaxation == SOCP
        W = model[Symbol("W")]
        s = x -> x+length(Buses)
        sqrd_volt = [W[i,i]+W[s(i),s(i)] for i=1:length(Buses)]
    end
    
    for i=1:length(Buses)
        constraint_power_balance!(model, Buses, Generators, Branches, sqrd_volt[i], i)
        voltage_bounds_sqrd!(model, Buses, sqrd_volt[i], i)
    end
    
    for i=1:length(Buses)
        for (j,l) in zip(From_nodes(i,Branches,Buses)...)
            if relaxation == NL
                sum_product_voltages  = VR[i]*VR[j]+VI[i]*VI[j]
                diff_product_voltages = VI[i]*VR[j]-VR[i]*VI[j]
            elseif relaxation == MC || relaxation == SDP || relaxation == SOCP
                sum_product_voltages  = W[i,j]+W[s(i),s(j)]
                diff_product_voltages = W[s(j),i]-W[s(i),j]
            end
            constraint_line_balance!(model, YR[l], YI[l], B[l], sqrd_volt[i], sqrd_volt[j], sum_product_voltages,  diff_product_voltages, i, j, l)
            constraint_phase_angle_diff!(model, ang_max, ang_min, sum_product_voltages, diff_product_voltages, i, j, l)
            line_thermal_bounds_sqrd!(model,thermal_lim, i, j, l)
        end
    end
    return model
end

: 

# Tests

## 5 buses

$|\mathcal{B}| = 5 $

$|\mathcal{L}| = 6 $

$|\mathcal{G}| = 5 $

In [ ]:
case_test = case_tests[2]

: 

In [ ]:
optimal_value = case_test[2][1]
filepath = joinpath(full_data,case_test[1]*".m")

: 

### Non linear - IPOPT

In [ ]:
model_nl = build_problem(filepath,ipopt,NL);

: 

In [ ]:
optimize!(model_nl)

: 

In [ ]:
optimality_gap(model_nl,optimal_value)

: 

### McCormick Relaxation - MOSEK

In [ ]:
model_mc = build_problem(filepath,mosek,MC);

: 

In [ ]:
optimize!(model_mc)

: 

In [ ]:
optimality_gap_mosek(model_mc,optimal_value,filepath)

: 

### Semidefinite Relaxation - MOSEK

In [ ]:
model_sdp = build_problem(filepath,mosek,SDP);

: 

In [ ]:
optimize!(model_sdp)

: 

In [ ]:
optimality_gap_mosek(model_sdp,optimal_value,filepath)

: 

### Second Order Cone Relaxation - MOSEK

In [ ]:
model_socp = build_problem(filepath,mosek,SOCP);

: 

In [ ]:
optimize!(model_socp)

: 

In [ ]:
optimality_gap_mosek(model_socp,optimal_value,filepath)

: 

### Results

In [ ]:
models = [model_nl,model_mc,model_sdp,model_socp];

: 

In [ ]:
function plot_var(models,variable,title,y_label)
    fig = figure(1)
    ax = fig.add_subplot(1,1,1)
    vars = [value.(model[Symbol(variable)]) for model in models]
    var_length = length(vars[1])
    models_type = ["NL","MC","SDP","SOCP"]
    width = 0.15
    G_labels = ["G"*string(i) for i = 1:var_length]
    xs = 1:var_length
    for (i,var) in enumerate(vars)
        ax.bar(xs.+(i-2.5)*width,var,width,label = models_type[i])
    end
    ax.set_title(title)
    ax.set_ylabel(y_label)
    ax.set_xticks(xs)
    ax.set_xticklabels(G_labels)
    ax.legend()
    return fig
end

: 

In [ ]:
plot_var(models,"p_g","Active Generation","Generation (MW)");

: 

In [ ]:
plot_var(models,"q_g","Reactive Generation","Generation (MW)");

: 

## 39 buses

$|\mathcal{B}| = 39 $

$|\mathcal{L}| = 46 $

$|\mathcal{G}| = 10 $

In [ ]:
case_test = case_tests[7]

: 

In [ ]:
filepath = joinpath(full_data,case_test[1]*".m")
optimal_value = case_test[2][1]

: 

### Non linear - IPOPT

In [ ]:
model_nl = build_problem(filepath,ipopt,NL);

: 

In [ ]:
optimize!(model_nl)

: 

In [ ]:
optimality_gap(model_nl,optimal_value)

: 

### McCormick Relaxation - MOSEK

In [ ]:
model_mc = build_problem(filepath,mosek,MC);

: 

In [ ]:
optimize!(model_mc)

: 

In [ ]:
optimality_gap_mosek(model_mc,optimal_value,filepath)

: 

### Semidefinite Relaxation - MOSEK

In [ ]:
model_sdp = build_problem(filepath,mosek,SDP);

: 

In [ ]:
optimize!(model_sdp)

: 

In [ ]:
optimality_gap_mosek(model_sdp,optimal_value,filepath)

: 

### Second Order Cone Relaxation - MOSEK

In [ ]:
model_socp = build_problem(filepath,mosek,SOCP);

: 

In [ ]:
optimize!(model_socp)

: 

In [ ]:
optimality_gap_mosek(model_socp,optimal_value,filepath)

: 

### Results

In [ ]:
models = [model_nl,model_mc,model_sdp,model_socp];

: 

In [ ]:
plot_var(models,"p_g","Active Generation","Generation (MW)");

: 

In [ ]:
plot_var(models,"q_g","Reactive Generation","Generation (MW)");

: 

## 162 buses

$|\mathcal{B}| = 162 $

$|\mathcal{L}| = 284 $

$|\mathcal{G}| = 12 $

In [ ]:
case_test = case_tests[12]

: 

In [ ]:
filepath = joinpath(full_data,case_test[1]*".m")
optimal_value = case_test[2][1]

: 

### Non linear - IPOPT

In [ ]:
model_nl = build_problem(filepath,ipopt,NL);

: 

In [ ]:
optimize!(model_nl)

: 

In [ ]:
optimality_gap(model_nl,optimal_value)

: 

### McCormick Relaxation - MOSEK

In [ ]:
model_mc = build_problem(filepath,mosek,MC);

: 

In [ ]:
optimize!(model_mc)

: 

In [ ]:
optimality_gap_mosek(model_mc,optimal_value,filepath)

: 

### Semidefinite Relaxation - MOSEK

In [ ]:
model_sdp = build_problem(filepath,mosek,SDP);

: 

In [ ]:
optimize!(model_sdp)

: 

In [ ]:
optimality_gap_mosek(model_sdp,optimal_value,filepath)

: 

### Second Order Cone Relaxation - MOSEK

In [ ]:
model_socp = build_problem(filepath,mosek,SOCP);

: 

In [ ]:
optimize!(model_socp)

: 

In [ ]:
optimality_gap_mosek(model_socp,optimal_value,filepath)

: 

### Results

In [ ]:
models = [model_nl,model_mc,model_sdp,model_socp];

: 

In [ ]:
plot_var(models,"p_g","Active Generation","Generation (MW)");

: 

In [ ]:
plot_var(models,"q_g","Reactive Generation","Generation (MW)");

: 

# Benchmark Table

## $|\mathcal{B}| \leq 500 $

In [ ]:
tests = case_tests[1:17];

: 

### Typical Operating Conditions

In [ ]:
cases = benchmark(tests);

: 

In [ ]:
print_cases_table(cases)

: 

In [ ]:
graph_dispersion(cases)

: 

### Congested Operating Conditions

In [ ]:
cases = benchmark(tests,API);

: 

In [ ]:
print_cases_table(cases)

: 

In [ ]:
graph_dispersion(cases)

: 

### Small Angle Difference Conditions

In [ ]:
cases = benchmark(tests,SAD);

: 

In [ ]:
print_cases_table(cases)

: 

In [ ]:
graph_dispersion(cases)

: 

## $588 \leq |\mathcal{B}| \leq 2000 $

In [ ]:
remove = [19]
range = filter(x -> !(x in remove),18:23);

: 

In [ ]:
tests = case_tests[range]

: 

### Typical Operating Conditions

In [ ]:
cases_no_sdp = benchmark(tests,TYP,false);

: 

In [ ]:
print_cases_no_sdp_table(cases_no_sdp)

: 

In [ ]:
graph_dispersion(cases_no_sdp,sdp=false)

: 

### Congested Operating Conditions

In [ ]:
cases_no_sdp = benchmark(tests,API,false);

: 

In [ ]:
print_cases_no_sdp_table(cases_no_sdp)

: 

In [ ]:
graph_dispersion(cases_no_sdp,sdp=false)

: 

### Small Angle Difference Conditions

In [ ]:
cases_no_sdp = benchmark(tests,SAD,false);

: 

In [ ]:
print_cases_no_sdp_table(cases_no_sdp)

: 

In [ ]:
graph_dispersion(cases_no_sdp,sdp=false)

: 